In [1]:
from __future__ import print_function
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
#from lib.envs.gridworld import GridworldEnv
from GridWorldEnv import GridworldEnv

In [2]:
env = GridworldEnv()

In [3]:
# option 1: NumPy version that assumes that P[s][a] has only one entry
next_state = np.empty((env.nS, env.nA), dtype=np.int)
reward = np.empty((env.nS, env.nA))
for st in range(env.nS):
    for ac in range(env.nA):
        _, next_state[st, ac], reward[st, ac], _ = env.P[st][ac][0]

# option 2: plain non-NumPy version
# end of options

In [4]:
# Taken from Policy Evaluation Exercise!

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation one our value function change is less than theta for all states.
        discount_factor: lambda discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        oldV = np.copy(V) # or oldV = V

        # option 1
        action_value = policy * 1.0 * (reward + discount_factor * V[next_state])
        V = np.sum(action_value, axis = 1)

        # option 2
        #V = np.zeros(env.nS)
        #for st in range(env.nS):
        #    for ac in range(env.nA):
        #        for pr, ns, rw, _ in env.P[st][ac]:
        #            V[st] += policy[st, ac] * pr * \
        #                                      (rw + discount_factor * oldV[ns])
        # end of options

        if np.max(np.abs(V - oldV)) < theta:
            break
    return V

In [5]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: Lambda discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        old_policy = np.copy(policy) # or old_policy = policy
        V = policy_eval_fn(policy, env, discount_factor)

        # option 1
        action_value = 1.0 * (reward + discount_factor * V[next_state])
        argmax = np.where(
                   action_value == np.max(action_value, axis = 1)[:,np.newaxis],
                   1.0, 0.0)
        policy = argmax / np.count_nonzero(argmax, axis = 1)[:,np.newaxis]

        # option 2 (omitted)
        # end of options

        if np.allclose(old_policy, policy, 1e-4):
            break
    return policy, V

In [6]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print()

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print()

print("Value Function:")
print(v)
print()

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print()



Policy Probability Distribution:
[[ 0.25  0.25  0.25  0.25]
 [ 0.    0.    0.    1.  ]
 [ 0.    0.    0.    1.  ]
 [ 0.    0.    0.5   0.5 ]
 [ 1.    0.    0.    0.  ]
 [ 0.5   0.    0.    0.5 ]
 [ 0.25  0.25  0.25  0.25]
 [ 0.    0.    1.    0.  ]
 [ 1.    0.    0.    0.  ]
 [ 0.25  0.25  0.25  0.25]
 [ 0.    0.5   0.5   0.  ]
 [ 0.    0.    1.    0.  ]
 [ 0.5   0.5   0.    0.  ]
 [ 0.    1.    0.    0.  ]
 [ 0.    1.    0.    0.  ]
 [ 0.25  0.25  0.25  0.25]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [7]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)